# 🐍 Join Datasets Notebook Overview
This notebook processes and merges IMDb and TMDB datasets, applying filtering and cleanup.

## 📂 Steps in This Notebook
1️⃣ **Unpacked IMDb Data** → Extracted GZIP files (one-time setup).  
2️⃣ **Loaded IMDb Title Basics** → Dropped unnecessary columns for efficiency.  
3️⃣ **Merged IMDb with Ratings** → Filtered for movies with **>1,000 votes**.  
4️⃣ **Merged Subgenres** → ⚠️ *Issue:* *Gladiator (2000)* got TMDB data from *Gladiator (1992)* (Needs Fixing).  
5️⃣ **Exported Titles for TMDB Data Retrieval** → Processed in `TMDBDataExtractor.ipynb`.  
6️⃣ **Imported & Merged TMDB Data** → Combined IMDb & TMDB metadata.  
7️⃣ **Final Filtering** → Kept **English language** movies with **revenue > $500K** (from **11,072 → 5,440** movies).  

## 📊 Current Dataset Columns
`tconst`, `title`, `year`, `runtime_minutes`, `genres`, `rating`, `numVotes`, `subgenres`, `budget`,  
`TMDB_id`, `origin_country`, `language`, `revenue`, `keywords`, `production_companies`, `cast`, `crew`.

##  🛠️ Next Steps
✔ **Fix *Gladiator* subgenre mismatch**  
✔ **Drop redundant columns (`title_x`, `title_y`, `genres_x`, `genres_y`)**  
✔ **Prepare dataset for graph-building & analysis**  

---

### Unpack (skip this)

In [ ]:
import gzip, shutil
import pandas as pd

with gzip.open('IMDb/new/title.basics.tsv.gz', 'rb') as f_in:
    with open('IMDb/new/title.basics.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('IMDb/new/title.ratings.tsv.gz', 'rb') as f_in:
    with open('IMDb/new/title.ratings.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('IMDb/new/name.basics.tsv.gz', 'rb') as f_in:
    with open('IMDb/new/name.basics.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('IMDb/new/title.akas.tsv.gz', 'rb') as f_in:
    with open('IMDb/new/title.akas.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('IMDb/new/title.crew.tsv.gz', 'rb') as f_in:
    with open('IMDb/new/title.crew.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### Load title_basics data
then filter by year to reduce rows

In [12]:
import pandas as pd

df_title_basics = pd.read_csv('../data/IMDb/new/title.basics.tsv', sep='\t', low_memory=False, na_values=['\\N'])
df_title_basics = df_title_basics.rename(columns={"startYear": "year"})

# Keep only rows where 'runtime' contains valid integers
df_title_basics["runtimeMinutes"] = df_title_basics["runtimeMinutes"].fillna("")
df_filtered = df_title_basics[df_title_basics["runtimeMinutes"].str.isdigit()].copy()
df_filtered["runtimeMinutes"] = df_filtered["runtimeMinutes"].astype(int)

df_title_basics_filtered = df_filtered[(df_filtered["year"] >= 1970) &
                                           (df_filtered["titleType"] == "movie") &
                                           (df_filtered["runtimeMinutes"] >= 60) &
                                           (df_filtered["runtimeMinutes"] <= 300)]

df_title_basics_filtered = df_title_basics_filtered.drop(columns=["isAdult", "endYear", "titleType"])
df_title_basics_filtered["year"] = df_title_basics_filtered["year"].astype(int)

df_title_basics_filtered.head()

,tconst,primaryTitle,originalTitle,year,runtimeMinutes,genres
15479,tt0015724,Dama de noche,Dama de noche,1993,102,"Drama,Mystery,Romance"
34794,tt0035423,Kate & Leopold,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance"
35957,tt0036606,"Another Time, Another Place","Another Time, Another Place",1983,118,"Drama,War"
38749,tt0039442,"Habla, mudita","Habla, mudita",1973,88,Drama
44149,tt0044952,Nagarik,Nagarik,1977,127,Drama


### MERGE BASICS WITH RATINGS
then filter by numVotes to further filter

In [13]:
df_title_ratings = pd.read_csv('../data/IMDb/new/title.ratings.tsv', sep='\t', low_memory=False, na_values=['\\N'])

df_merged = pd.merge(df_title_basics_filtered, df_title_ratings, on="tconst", how="inner")

# FILTER FOR POPULAR MOVIES
df_merged_filtered = df_merged[df_merged["numVotes"] > 1000]

print(f"Rows: {len(df_merged_filtered)}")
df_merged_filtered.head()

Rows: 39339


,tconst,primaryTitle,originalTitle,year,runtimeMinutes,genres,averageRating,numVotes
1,tt0035423,Kate & Leopold,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance",6.4,91466
6,tt0054724,I Eat Your Skin,Zombie,1971,92,Horror,3.6,1733
24,tt0061592,Doomsday Machine,Doomsday Machine,1976,83,Sci-Fi,2.6,1450
30,tt0062690,The Awakening of the Beast,O Ritual dos Sádicos,1970,93,"Drama,Horror",5.9,1368
43,tt0063142,Isle of the Snake People,La muerte viviente,1971,90,"Horror,Mystery",3.4,1095


### MERGE THOSE WITH SUB-GENRES

In [14]:
import ast

df_subgenres = pd.read_csv("../data/IMDb/titles_subgenres.csv")
print(df_subgenres.columns)
df_merged2 = df_subgenres.merge(df_merged_filtered, how="inner", left_on=["title", "year"], right_on=["primaryTitle", "year"])

# CLEAN
df_merged2["subgenres"] = df_merged2["subgenres"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_merged2["subgenres_str"] = df_merged2["subgenres"].apply(lambda x: ", ".join(x))
df_merged2 = df_merged2.drop(columns=["subgenres", "primaryTitle"])
df_merged2 = df_merged2.rename(columns={"subgenres_str": "subgenres"})

print(f"Rows: {len(df_merged2)}")
df_merged2.head()

Index(['title', 'year', 'subgenres'], dtype='object')
Rows: 9491


,title,year,tconst,originalTitle,runtimeMinutes,genres,averageRating,numVotes,subgenres
0,Captain America: Brave New World,2025.0,tt14513804,Captain America: Brave New World,118,"Action,Adventure,Sci-Fi",5.9,75784,"action-epic, epic-adventure, epic-sci-fi, supe..."
1,Gladiator II,2024.0,tt9218128,Gladiator II,148,"Action,Adventure,Drama",6.6,215175,"action-epic, epic-adventure, epic-drama, perio..."
2,Furiosa: A Mad Max Saga,2024.0,tt12037194,Furiosa: A Mad Max Saga,148,"Action,Adventure,Sci-Fi",7.5,279232,"action-epic, car-action, desert-adventure, dys..."
3,Dune: Part Two,2024.0,tt15239678,Dune: Part Two,166,"Action,Adventure,Drama",8.5,611472,"action-epic, desert-adventure, epic-drama, epi..."
4,Gladiator,2000.0,tt0172495,Gladiator,155,"Action,Adventure,Drama",8.5,1742974,"action-epic, epic-adventure, epic-drama, perio..."


### EXPORT TITLES FOR FURTHER DATA AQUISITION FROM TMDB

In [4]:
df_merged2[["title", "tconst"]].to_csv("titles_with_subgenres.csv")
print("saved")

saved


### IMPORT JSON DATA

In [15]:
df_tmdb = pd.read_csv("../data/cache/parsed_json_data.csv")
df_tmdb = df_tmdb.rename(columns={"id": "tmdb_id"})
df_tmdb = df_tmdb.drop(["original_title", "keywords", "title", "origin_country", "genres"], axis=1)
print(f"Rows: {len(df_tmdb)}")
df_tmdb.head()

Rows: 11072


,budget,original_language,revenue,production_companies,cast,crew,franchise,tconst
0,48000000,en,76019048,"['Konrad Pictures', 'Miramax']","[{'name': 'Meg Ryan', 'order': 0, 'character':...","[{'name': 'James Mangold', 'job': 'Director', ...",False,tt0035423
1,0,fr,0,"['Les Films La Boétie', 'Euro International Fi...","[{'name': 'Stéphane Audran', 'order': 0, 'char...","[{'name': 'Claude Chabrol', 'job': 'Director',...",False,tt0064106
2,0,en,0,"['Roxanne Company', 'American International Pi...","[{'name': 'Shirley Stoler', 'order': 0, 'chara...","[{'name': 'Leonard Kastle', 'job': 'Director',...",False,tt0064437
3,0,en,0,"['Triumvirate Films', 'United Artists']","[{'name': 'Jenny Agutter', 'order': 0, 'charac...","[{'name': 'David Greene', 'job': 'Director', '...",False,tt0064462
4,0,cs,0,['Filmové studio Barrandov'],"[{'name': 'Elo Romančík', 'order': 0, 'charact...","[{'name': 'Otakar Vávra', 'job': 'Director', '...",False,tt0064546


### MERGE IMDB & TMDB DATA (CROSSING THE STREAMS)

In [16]:
def merge_tmdb_with_imdb(df_imdb, df_tmdb):
    """Merge TMDB JSON data into the IMDb dataset."""
    # Merge IMDb dataset with extracted TMDB data
    df_merged = pd.merge(df_imdb, df_tmdb, on="tconst")
    
    return df_merged

# Merge TMDB data into IMDb dataset (df_merged2)
df_final = merge_tmdb_with_imdb(df_merged2, df_tmdb)


#df_final.to_csv("merged_imdb_tmdb.csv", index=False)

df_final.tail(1)

,title,year,tconst,originalTitle,runtimeMinutes,genres,averageRating,numVotes,subgenres,budget,original_language,revenue,production_companies,cast,crew,franchise
9487,Chrysalis,2014.0,tt2836260,Chrysalis,100,"Drama,Horror,Sci-Fi",4.7,1008,zombie-horror,0,en,0,"['CNGM Pictures', 'FOUR Productions', 'Glass C...","[{'name': 'Sara Gorsky', 'order': 0, 'characte...","[{'name': 'John Klein', 'job': 'Director', 'ge...",False


### FILTER

In [17]:
filtered = df_final[
    (df_final["revenue"] >= 500_000) &
    (df_final["original_language"] == "en") &
    (df_final["budget"] > 0)
].copy()

filtered = filtered.drop(["original_language"], axis=1)

rows_after = filtered.shape[0]
rows_before = df_final.shape[0]
print(f"Before: {rows_before}\nAfter: {rows_after}")
filtered.columns

Before: 9488
After: 4688


Index(['title', 'year', 'tconst', 'originalTitle', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'subgenres', 'budget', 'revenue', 'production_companies', 'cast', 'crew', 'franchise'], dtype='object')

In [8]:
import pandas as pd

# Assuming your DataFrame is called df
missing_counts = filtered.isna().sum()

# Display only columns that have at least one missing value
#missing_counts = missing_counts[missing_counts > 0]

print(missing_counts)


Unnamed: 0              0
title                   0
tconst                  0
budget                  0
revenue                 0
production_companies    0
cast                    0
crew                    0
franchise               0
dtype: int64


### EXPLODE DATA FOR GRAPH

In [18]:
import pandas as pd
import ast

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df = filtered.copy()

# Create a list of flat relationship rows
flat_rows = []
n_cast = 20

for _, row in df.iterrows():
    # Unpack movie info
    from_movie = {
        'from_label': 'Movie',
        'from_tconst': row['tconst'],
        'from_title': row['title'],
        'from_year': row['year'],
        'from_runtimeMinutes': row['runtimeMinutes'],
        'from_averageRating': row['averageRating'],
        'from_numVotes': row['numVotes'],
        'from_budget': row['budget'],
        'from_revenue': row['revenue'],
        'from_franchise': row['franchise']
    }

    # === GENRES ===
    for genre in row['genres'].split(','):
        flat_rows.append({
            **from_movie,
            'relationship': 'IN_GENRE',
            'to_label': 'Genre',
            'to_name': genre.strip(),
            'to_id': None, 'to_gender': None, 'to_popularity': None, 'to_adult': None
        })

    # === SUBGENRES ===
    for subgenre in row['subgenres'].split(', '):
        flat_rows.append({
            **from_movie,
            'relationship': 'HAS_SUBGENRE',
            'to_label': 'Subgenre',
            'to_name': subgenre.strip(),
            'to_id': None, 'to_gender': None, 'to_popularity': None, 'to_adult': None
        })

    # === PRODUCTION COMPANIES ===
    companies = ast.literal_eval(row['production_companies']) if isinstance(row['production_companies'], str) else row['production_companies']
    for company in companies:
        flat_rows.append({
            **from_movie,
            'relationship': 'PRODUCED_BY',
            'to_label': 'ProductionCompany',
            'to_name': company,
            'to_id': None, 'to_gender': None, 'to_popularity': None, 'to_adult': None
        })

    # === CAST (Actors) ===
    cast_list = ast.literal_eval(row['cast']) if isinstance(row['cast'], str) else row['cast']
    for actor in cast_list:
        if int(actor.get('order', 999)) <= n_cast:
            flat_rows.append({
                **from_movie,
                'relationship': 'ACTED_IN',
                'to_label': 'Actor',
                'to_name': actor.get('name'),
                'to_id': actor.get('id'),
                'to_gender': actor.get('gender'),
                'to_popularity': actor.get('popularity'),
                'to_adult': actor.get('adult')
            })

    # === CREW (Directors, Producers, DP) ===
    relevant_jobs = {'Director': 'DIRECTED', 'Producer': 'PRODUCED', 'Director of Photography': 'PHOTOGRAPHED'}
    crew_list = ast.literal_eval(row['crew']) if isinstance(row['crew'], str) else row['crew']
    for crew in crew_list:
        job = crew.get('job')
        if job in relevant_jobs:
            flat_rows.append({
                **from_movie,
                'relationship': relevant_jobs[job],
                'to_label': job,
                'to_name': crew.get('name'),
                'to_id': crew.get('id'),
                'to_gender': crew.get('gender'),
                'to_popularity': crew.get('popularity'),
                'to_adult': None  # Crew members don't have this field
            })

# === Create flat DataFrame ===
exploded_df = pd.DataFrame(flat_rows)

# Optional: Fill missing values with empty strings to avoid NaNs in CSV
exploded_df = exploded_df.fillna('')

# === Save as Neo4j-ready CSV ===
exploded_df.to_csv('graph_data.csv', index=False)

# Preview first rows
exploded_df.head()


,from_label,from_tconst,from_title,from_year,from_runtimeMinutes,from_averageRating,from_numVotes,from_budget,from_revenue,from_franchise,relationship,to_label,to_name,to_id,to_gender,to_popularity,to_adult
0,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000,388056272,True,IN_GENRE,Genre,Action,,,,
1,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000,388056272,True,IN_GENRE,Genre,Adventure,,,,
2,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000,388056272,True,IN_GENRE,Genre,Sci-Fi,,,,
3,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000,388056272,True,HAS_SUBGENRE,Subgenre,action-epic,,,,
4,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000,388056272,True,HAS_SUBGENRE,Subgenre,epic-adventure,,,,


### UNEXPLODE DATA

In [20]:
import pandas as pd

df = pd.read_csv("../data/cache/graph_data.csv")

# === Aggregate genres and subgenres ===
genre_df = df[df["to_label"] == "Genre"].groupby("from_tconst")["to_name"].apply(
    lambda x: ', '.join(sorted(set(x)))
).reset_index(name="genres")

subgenre_df = df[df["to_label"] == "Subgenre"].groupby("from_tconst")["to_name"].apply(
    lambda x: ', '.join(sorted(set(x)))
).reset_index(name="subgenres")

# === Aggregate production companies ===
company_df = df[df["to_label"] == "ProductionCompany"].groupby("from_tconst")["to_name"].apply(
    lambda x: sorted(set(x))
).reset_index(name="production_companies")

# === Aggregate cast ===
actor_df = df[df["to_label"] == "Actor"].groupby("from_tconst", group_keys=False).apply(
    lambda group: [
        {
            "name": row["to_name"],
            "id": row["to_id"],
            "gender": row["to_gender"],
            "popularity": row["to_popularity"],
            "order": i  # Not perfect, but preserves order
        }
        for i, row in group.iterrows()
    ]
).reset_index(name="cast")

# === Aggregate crew ===
crew_jobs = ["Director", "Producer", "Director of Photography"]
crew_df = df[df["to_label"].isin(crew_jobs)].groupby("from_tconst", group_keys=False).apply(
    lambda group: [
        {
            "name": row["to_name"],
            "id": row["to_id"],
            "gender": row["to_gender"],
            "popularity": row["to_popularity"],
            "job": row["to_label"]
        }
        for _, row in group.iterrows()
    ]
).reset_index(name="crew")

# === Get unique movie metadata ===
movie_metadata = df.groupby("from_tconst").first().reset_index()

# === Merge everything together ===
df_unexploded = movie_metadata.merge(genre_df, on="from_tconst", how="left")
df_unexploded = df_unexploded.merge(subgenre_df, on="from_tconst", how="left")
df_unexploded = df_unexploded.merge(company_df, on="from_tconst", how="left")
df_unexploded = df_unexploded.merge(actor_df, on="from_tconst", how="left")
df_unexploded = df_unexploded.merge(crew_df, on="from_tconst", how="left")

# Optional: drop Neo4j-style metadata columns not needed anymore
df_unexploded = df_unexploded.drop(columns=["to_name", "to_label", "relationship"], errors='ignore')

# Save it
df_unexploded.to_csv("unexploded_data.csv", index=False)

print("✅ Reconstructed wide-format dataset with actors, crew, genres, companies.")
df_unexploded.head()

C:\Users\Joe\AppData\Local\Temp\ipykernel_15476\138991361.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  actor_df = df[df["to_label"] == "Actor"].groupby("from_tconst", group_keys=False).apply(
C:\Users\Joe\AppData\Local\Temp\ipykernel_15476\138991361.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  crew_df = df[df["to_label"].isin(crew_jobs)].groupby("from_tconst", group_keys=False).apply(


✅ Reconstructed wide-format dataset with actors, crew, genres, companies.


,from_tconst,from_label,from_title,from_year,from_runtimeMinutes,from_averageRating,from_numVotes,from_budget,from_revenue,to_id,to_gender,to_popularity,profit,profit_percent_gain,from_budget_category,from_budget_adj_new,from_revenue_adj_new,profit_new,profit_percent_gain_new,from_budget_category_new,from_budget_adj_new.1,from_revenue_adj_new.1,profit_new.1,profit_percent_gain_new.1,from_budget_category_new.1,genres,subgenres,production_companies,cast,crew
0,tt0035423,Movie,Kate & Leopold,2001.0,118,6.4,91466,8.258439e+07,1.307914e+08,5344.0,1.0,0.094,4.820700e+07,58.373017,high,8.258439e+07,1.307914e+08,4.820700e+07,58.373017,high,8.258439e+07,1.307914e+08,4.820700e+07,58.373017,high,"Comedy, Fantasy, Romance",feel-good-romance,"[Konrad Pictures, Miramax]","[{'name': 'Meg Ryan', 'id': 5344.0, 'gender': ...","[{'name': 'James Mangold', 'id': 366.0, 'gende..."
1,tt0065134,Movie,Two Mules for Sister Sara,1970.0,116,7.0,31918,1.963286e+07,3.965838e+07,4090.0,1.0,0.065,2.002552e+07,102.000000,low,1.963286e+07,3.965838e+07,2.002552e+07,102.000000,low,1.963286e+07,3.965838e+07,2.002552e+07,102.000000,low,"Adventure, Drama, Romance",quest-adventure,"[Malpaso Productions, Martin Rackin Production...","[{'name': 'Shirley MacLaine', 'id': 4090.0, 'g...","[{'name': 'Don Siegel', 'id': 14773.0, 'gender..."
2,tt0065377,Movie,Airport,1970.0,137,6.6,22999,7.853144e+07,7.891558e+08,13784.0,2.0,0.059,7.106244e+08,904.891510,high,7.853144e+07,7.891558e+08,7.106244e+08,904.891510,high,7.853144e+07,7.891558e+08,7.106244e+08,904.891510,high,"Action, Drama, Thriller","dark-comedy, disaster-action",[Ross Hunter Productions],"[{'name': 'Burt Lancaster', 'id': 13784.0, 'ge...","[{'name': 'George Seaton', 'id': 24939.0, 'gen..."
3,tt0065462,Movie,Beneath the Planet of the Apes,1970.0,95,6.0,55550,2.355943e+07,1.492075e+08,18643.0,2.0,0.025,1.256481e+08,533.323933,medium,2.355943e+07,1.492075e+08,1.256481e+08,533.323933,medium,2.355943e+07,1.492075e+08,1.256481e+08,533.323933,medium,"Action, Adventure, Sci-Fi",dystopian-sci-fi,"[20th Century Fox, APJAC Productions]","[{'name': 'James Franciscus', 'id': 18643.0, '...","[{'name': 'Ted Post', 'id': 18635.0, 'gender':..."
4,tt0065466,Movie,Beyond the Valley of the Dolls,1970.0,109,6.1,12467,1.641307e+07,7.067830e+07,45110.0,1.0,0.012,5.426523e+07,330.622010,low,1.641307e+07,7.067830e+07,5.426523e+07,330.622010,low,1.641307e+07,7.067830e+07,5.426523e+07,330.622010,low,"Comedy, Drama, Music","parody-comedy, satire-comedy",[20th Century Fox],"[{'name': 'Dolly Read', 'id': 45110.0, 'gender...","[{'name': 'Russ Meyer', 'id': 4590.0, 'gender'..."


### ADJUST FOR INFLATION

In [21]:
import pandas as pd
import cpi

# Make sure CPI data is loaded7
#cpi.update()

# Adjust budget to 2023 dollars
#df_unexploded = pd.read_csv("unexploded_data.csv")

# Convert to numeric first, forcing errors to NaN, then dropna and cast to int
df_unexploded["from_year"] = pd.to_numeric(df_unexploded["from_year"], errors="coerce").dropna().astype(int)
df_unexploded["from_budget"] = pd.to_numeric(df_unexploded["from_budget"], errors="coerce")
df_unexploded["from_revenue"] = pd.to_numeric(df_unexploded["from_revenue"], errors="coerce")
df_unexploded["from_year_adj"] = df_unexploded["from_year"].clip(upper=2023).astype(int)

df_unexploded["from_budget_adj"] = df_unexploded.apply(
    lambda row: cpi.inflate(row["from_budget"], row["from_year_adj"], to=2023) if pd.notnull(row["from_budget"]) else None,
    axis=1
)

# Adjust revenue to 2023 dollars
df_unexploded["from_revenue_adj"] = df_unexploded.apply(
    lambda row: cpi.inflate(row["from_revenue"], row["from_year_adj"], to=2023) if pd.notnull(row["from_revenue"]) else None,
    axis=1
)

df_unexploded.head()

,from_tconst,from_label,from_title,from_year,from_runtimeMinutes,from_averageRating,from_numVotes,from_budget,from_revenue,to_id,to_gender,to_popularity,profit,profit_percent_gain,from_budget_category,from_budget_adj_new,from_revenue_adj_new,profit_new,profit_percent_gain_new,from_budget_category_new,from_budget_adj_new.1,from_revenue_adj_new.1,profit_new.1,profit_percent_gain_new.1,from_budget_category_new.1,genres,subgenres,production_companies,cast,crew,from_year_adj,from_budget_adj,from_revenue_adj
0,tt0035423,Movie,Kate & Leopold,2001,118,6.4,91466,8.258439e+07,1.307914e+08,5344.0,1.0,0.094,4.820700e+07,58.373017,high,8.258439e+07,1.307914e+08,4.820700e+07,58.373017,high,8.258439e+07,1.307914e+08,4.820700e+07,58.373017,high,"Comedy, Fantasy, Romance",feel-good-romance,"[Konrad Pictures, Miramax]","[{'name': 'Meg Ryan', 'id': 5344.0, 'gender': ...","[{'name': 'James Mangold', 'id': 366.0, 'gende...",2001,1.420871e+08,2.250277e+08
1,tt0065134,Movie,Two Mules for Sister Sara,1970,116,7.0,31918,1.963286e+07,3.965838e+07,4090.0,1.0,0.065,2.002552e+07,102.000000,low,1.963286e+07,3.965838e+07,2.002552e+07,102.000000,low,1.963286e+07,3.965838e+07,2.002552e+07,102.000000,low,"Adventure, Drama, Romance",quest-adventure,"[Malpaso Productions, Martin Rackin Production...","[{'name': 'Shirley MacLaine', 'id': 4090.0, 'g...","[{'name': 'Don Siegel', 'id': 14773.0, 'gender...",1970,1.541797e+08,3.114430e+08
2,tt0065377,Movie,Airport,1970,137,6.6,22999,7.853144e+07,7.891558e+08,13784.0,2.0,0.059,7.106244e+08,904.891510,high,7.853144e+07,7.891558e+08,7.106244e+08,904.891510,high,7.853144e+07,7.891558e+08,7.106244e+08,904.891510,high,"Action, Drama, Thriller","dark-comedy, disaster-action",[Ross Hunter Productions],"[{'name': 'Burt Lancaster', 'id': 13784.0, 'ge...","[{'name': 'George Seaton', 'id': 24939.0, 'gen...",1970,6.167188e+08,6.197354e+09
3,tt0065462,Movie,Beneath the Planet of the Apes,1970,95,6.0,55550,2.355943e+07,1.492075e+08,18643.0,2.0,0.025,1.256481e+08,533.323933,medium,2.355943e+07,1.492075e+08,1.256481e+08,533.323933,medium,2.355943e+07,1.492075e+08,1.256481e+08,533.323933,medium,"Action, Adventure, Sci-Fi",dystopian-sci-fi,"[20th Century Fox, APJAC Productions]","[{'name': 'James Franciscus', 'id': 18643.0, '...","[{'name': 'Ted Post', 'id': 18635.0, 'gender':...",1970,1.850156e+08,1.171748e+09
4,tt0065466,Movie,Beyond the Valley of the Dolls,1970,109,6.1,12467,1.641307e+07,7.067830e+07,45110.0,1.0,0.012,5.426523e+07,330.622010,low,1.641307e+07,7.067830e+07,5.426523e+07,330.622010,low,1.641307e+07,7.067830e+07,5.426523e+07,330.622010,low,"Comedy, Drama, Music","parody-comedy, satire-comedy",[20th Century Fox],"[{'name': 'Dolly Read', 'id': 45110.0, 'gender...","[{'name': 'Russ Meyer', 'id': 4590.0, 'gender'...",1970,1.288942e+08,5.550469e+08


| category               | lower bound (usd) | upper bound (usd) |
|------------------------|-------------------|-------------------|
| indie                    | 0        | 2M         |
| low                    | 2M         | 20M        |
| medium                 | 20M        | 50M        |
| high                   | 50M        | 100M       |
| tentpole               | 100M       | ∞          |

### DISCRITIZE

In [22]:
# discritize budget
# Define bin edges and labels
budget_bins = [2_000_000, 20_000_000, 50_000_000, 100_000_000, float("inf")]

budget_labels = [
    "low",
    "medium",
    "high",
    "tentpole"
]

# Apply discretization
df_unexploded["from_budget_category"] = pd.cut(
    df_unexploded["from_budget_adj"],
    bins=budget_bins,
    labels=budget_labels,
    right=False  # means intervals are [), e.g. 0 <= x < 20,000
)

print(df_unexploded["from_budget_category"].value_counts())

# discritize profit
df_unexploded['profit'] = df_unexploded['from_revenue_adj'] - df_unexploded['from_budget_adj']

df_unexploded['profit_percent_gain'] = 100 * df_unexploded['profit'] / df_unexploded['from_budget_adj']

df_unexploded.to_csv("../data/poster3_data.csv")

from_budget_category
tentpole    1827
high        1105
medium      1004
low          690
Name: count, dtype: int64


In [31]:
df_unexploded = df_unexploded.merge(
    df_tmdb[["tconst", "franchise"]],
    left_on="from_tconst",  # from df_unexploded
    right_on="tconst",      # from df_tmdb
    how="left"
).drop(columns=["tconst"])  # Optionally drop the duplicate key


df_unexploded.columns

Index(['from_tconst', 'from_label', 'from_title', 'from_year', 'from_runtimeMinutes', 'from_averageRating', 'from_numVotes', 'from_budget', 'from_revenue', 'to_id', 'to_gender', 'to_popularity', 'profit', 'profit_percent_gain', 'from_budget_category', 'from_budget_adj_new', 'from_revenue_adj_new', 'profit_new', 'profit_percent_gain_new', 'from_budget_category_new', 'from_budget_adj_new.1', 'from_revenue_adj_new.1', 'profit_new.1', 'profit_percent_gain_new.1', 'from_budget_category_new.1', 'genres', 'subgenres', 'production_companies', 'cast', 'crew', 'from_year_adj', 'from_budget_adj', 'from_revenue_adj', 'franchise'], dtype='object')

### TRAIN / TEST SPLIT

In [26]:
df_filtered2 = df[df["from_budget_category"].notna()].copy()
print("Before filtering:", len(df))
print("After filtering:", len(df_filtered2))

Before filtering: 4626
After filtering: 4626


In [35]:
from sklearn.model_selection import train_test_split
import os

# Ensure franchise column is present
df = df_unexploded[df_unexploded["from_budget_category"].notna()].copy()

# Separate franchise and non-franchise
franchise_df = df[df["franchise"] == True]
non_franchise_df = df[df["franchise"] == False]

# Sample exactly 232 non-franchise movies for the test set (stratified)
df_test = (
    non_franchise_df
    .groupby("from_budget_category", group_keys=False)
    .apply(lambda x: x.sample(frac=232 / len(non_franchise_df), random_state=42))
    .reset_index(drop=True)
)

# Remaining non-franchise movies go to training
df_train = non_franchise_df[~non_franchise_df["from_tconst"].isin(df_test["from_tconst"])]

# Add franchise movies to training set
df_train = pd.concat([df_train, franchise_df], ignore_index=True)

# Save to cache
cache_dir = os.path.join("..", "data", "Cache")
df_train.to_csv(os.path.join(cache_dir, "poster3_train.csv"), index=False)
df_test.to_csv(os.path.join(cache_dir, "poster3_test.csv"), index=False)

# Summary
print(f"Train size (with franchises): {len(df_train)}")
print(f"Test size (non-franchise only): {len(df_test)}")

C:\Users\Joe\AppData\Local\Temp\ipykernel_15476\1217811403.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("from_budget_category", group_keys=False)
C:\Users\Joe\AppData\Local\Temp\ipykernel_15476\1217811403.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=232 / len(non_franchise_df), random_state=42))


Train size (with franchises): 4395
Test size (non-franchise only): 231


### UPDATE LONG DATASET

In [47]:
import pandas as pd

# Load existing graph data (train+val long-format)
exploded_df = pd.read_csv("../data/cache/graph_data.csv")

# Columns to update from train/val
cols_to_update = ['from_budget_adj', 'from_revenue_adj', 'profit', 
                  'profit_percent_gain', 'from_budget_category']

# Add label to training/validation set
df_train['label'] = 'train'

# Merge updated info from train/val set
df_updated = exploded_df.merge(
    df_train[['from_tconst'] + cols_to_update + ['label']],
    on='from_tconst',
    how='left',
    suffixes=('', '_new')
)

# Update values where we have new info
df_updated['from_budget'] = df_updated['from_budget_adj']
df_updated['from_revenue'] = df_updated['from_revenue_adj']

# Drop temp columns
df_updated.drop(columns=['from_budget_adj', 'from_revenue_adj', 'to_adult'], inplace=True, errors='ignore')

# Prepare test set
df_test['label'] = 'test'
graph_cols = df_updated.columns.tolist()
df_test = df_test[[col for col in df_test.columns if col in graph_cols]]
df_test = df_test.reindex(columns=graph_cols)

# Combine updated train/val with test
df_combined = pd.concat([df_updated, df_test], ignore_index=True)

# Save
df_combined.to_csv("../data/cache/graph_data.csv", index=False)
df_combined.head()

,from_label,from_tconst,from_title,from_year,from_runtimeMinutes,from_averageRating,from_numVotes,from_budget,from_revenue,relationship,to_label,to_name,to_id,to_gender,to_popularity,profit,profit_percent_gain,from_budget_category,from_budget_adj_new,from_revenue_adj_new,profit_new,profit_percent_gain_new,from_budget_category_new,from_budget_adj_new.1,from_revenue_adj_new.1,profit_new.1,profit_percent_gain_new.1,from_budget_category_new.1,profit_new.2,profit_percent_gain_new.2,from_budget_category_new.2,profit_new,profit_percent_gain_new,from_budget_category_new,label
0,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000.0,388056272.0,IN_GENRE,Genre,Action,NaN,NaN,NaN,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,NaN,NaN,NaN,208056272.0,115.586818,tentpole,train
1,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000.0,388056272.0,IN_GENRE,Genre,Adventure,NaN,NaN,NaN,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,NaN,NaN,NaN,208056272.0,115.586818,tentpole,train
2,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000.0,388056272.0,IN_GENRE,Genre,Sci-Fi,NaN,NaN,NaN,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,NaN,NaN,NaN,208056272.0,115.586818,tentpole,train
3,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000.0,388056272.0,HAS_SUBGENRE,Subgenre,action-epic,NaN,NaN,NaN,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,NaN,NaN,NaN,208056272.0,115.586818,tentpole,train
4,Movie,tt14513804,Captain America: Brave New World,2025.0,118,5.9,75784,180000000.0,388056272.0,HAS_SUBGENRE,Subgenre,epic-adventure,NaN,NaN,NaN,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,180000000.0,388056272.0,208056272.0,115.586818,tentpole,NaN,NaN,NaN,208056272.0,115.586818,tentpole,train


### JOIN GRAPH FEATURES

In [31]:
import pandas as pd
import os

# Paths
cache_dir = os.path.join("..", "data", "cache")

# Load base splits
df_train = pd.read_csv(os.path.join(cache_dir, "poster3_train.csv"))
df_val = pd.read_csv(os.path.join(cache_dir, "poster3_val.csv"))

# Load graph features
df_bet = pd.read_csv("../data/from_graph/betweenness_centrality.csv")
df_deg = pd.read_csv("../data/from_graph/degree.csv")
df_comm = pd.read_csv("../data/from_graph/community_ids.csv")
df_comm_size = pd.read_csv("../data/from_graph/community_size.csv")

# Define a merge function
def merge_graph_features(df):
    df = df.merge(df_bet[['from_tconst', 'm.betweenness_m2m']], on='from_tconst', how='left')
    df = df.merge(df_deg[['from_tconst', 'm.degree']], on='from_tconst', how='left')
    df = df.merge(df_comm[['from_tconst', 'm.community_id']], on='from_tconst', how='left')
    df = df.merge(df_comm_size[['from_tconst', 'm.community_size']], on='from_tconst', how='left')
    df[['m.betweenness_m2m', 'm.degree', 'm.community_id', 'm.community_size']] = \
        df[['m.betweenness_m2m', 'm.degree', 'm.community_id', 'm.community_size']].fillna(0)
    return df

# Merge for train and val
df_train = merge_graph_features(df_train)
df_val = merge_graph_features(df_val)

# Save back to cache
df_train.to_csv(os.path.join(cache_dir, "poster3_train_graphed.csv"), index=False)
df_val.to_csv(os.path.join(cache_dir, "poster3_val_graphed.csv"), index=False)


In [44]:
df_test["from_title"].iloc[200:]

200                                     Robin Hood
201                                      Ferdinand
202                      My Stepmother Is an Alien
203                                    The Natural
204                                    The Mission
205        The Messenger: The Story of Joan of Arc
206                                A Perfect World
207    Talladega Nights: The Ballad of Ricky Bobby
208                                             Up
209                    Men in Black: International
210                                      The Score
211                           The Three Musketeers
212                                     Ghost Ship
213                                  Love Actually
214                                     Unfaithful
215                                Very Bad Things
216                            Alita: Battle Angel
217                              Death Becomes Her
218                               Private Benjamin
219                            